In [1]:
import pandas as pd
import json
from urllib.request import urlopen
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"

## Cases Data

In [2]:
# load the data
cases_raw = pd.read_excel('data/nga_subnational_covid19_hera.xlsx')
cases = cases_raw.copy()

In [3]:
cases.dtypes

ID                                        int64
DATE                             datetime64[ns]
ISO_3                                    object
PAYS                                     object
ID_PAYS                                   int64
REGION                                   object
ID_REGION                                 int64
CONTAMINES                              float64
DECES                                   float64
GUERIS                                  float64
CONTAMINES_FEMME                        float64
CONTAMINES_HOMME                        float64
CONTAMINES_GENRE_NON_SPECIFIE           float64
SOURCE                                   object
dtype: object

In [4]:
# change the date object to a datetime
#cases['date'] = pd.to_datetime(cases['date'])
# keep only the useful columns
cases = cases[['DATE', 'REGION', 'CONTAMINES']]
cases = cases[cases['REGION'] != 'Non spécifié']

In [5]:
cases

,DATE,REGION,CONTAMINES
0,2020-02-27,Abia,0.0
1,2020-02-27,Adamawa,0.0
2,2020-02-27,Akwa Ibom,0.0
3,2020-02-27,Anambra,0.0
4,2020-02-27,Bauchi,0.0
...,...,...,...
12458,2021-01-19,Rivers,44.0
12459,2021-01-19,Sokoto,0.0
12460,2021-01-19,Taraba,0.0
12461,2021-01-19,Yobe,0.0


In [6]:
# aggregate by week
state_cases_weekly = cases.groupby('REGION').resample('W-Mon', on='DATE').sum()\
                            .reset_index().rename(columns={'CONTAMINES': 'CASES'})

In [7]:
state_cases_weekly.head()

,REGION,DATE,CASES
0,Abia,2020-03-02,0.0
1,Abia,2020-03-09,0.0
2,Abia,2020-03-16,0.0
3,Abia,2020-03-23,0.0
4,Abia,2020-03-30,0.0


In [8]:
# pivot and then melt to get values for every week
# state_cases_weekly = state_cases_weekly.pivot(index='date', columns='province/state', values='cases') \
#                         .fillna(0).reset_index()

# state_cases_weekly = pd.melt(state_cases_weekly, id_vars='date', value_name='cases')
state_cases_weekly['PREV_WEEK_CASES'] = state_cases_weekly['CASES'].shift(1).fillna(0)
state_cases_weekly.tail()

,REGION,DATE,CASES,PREV_WEEK_CASES
1771,Zamfara,2020-12-28,8.0,0.0
1772,Zamfara,2021-01-04,25.0,8.0
1773,Zamfara,2021-01-11,47.0,25.0
1774,Zamfara,2021-01-18,3.0,47.0
1775,Zamfara,2021-01-25,3.0,3.0


In [9]:
# get weekly change
state_cases_weekly['weekly_change'] = (state_cases_weekly[['PREV_WEEK_CASES', 'CASES']]\
                                            .pct_change(axis=1)['CASES']).fillna(0)*100

In [10]:
state_cases_weekly.head()

,REGION,DATE,CASES,PREV_WEEK_CASES,weekly_change
0,Abia,2020-03-02,0.0,0.0,0.0
1,Abia,2020-03-09,0.0,0.0,0.0
2,Abia,2020-03-16,0.0,0.0,0.0
3,Abia,2020-03-23,0.0,0.0,0.0
4,Abia,2020-03-30,0.0,0.0,0.0


In [11]:
# aggregate at state level to get totals
state_cases_total = state_cases_weekly.groupby('REGION')['REGION','CASES'].sum().reset_index()
state_cases_total.head()

,REGION,CASES
0,Abia,1134.0
1,Adamawa,573.0
2,Akwa Ibom,698.0
3,Anambra,515.0
4,Bauchi,1107.0


In [12]:
# get the total national cases by week
national_cases_weekly = state_cases_weekly.groupby('DATE')['CASES'].sum().reset_index()
national_cases_weekly.tail()

,DATE,CASES
43,2020-12-28,6021.0
44,2021-01-04,6540.0
45,2021-01-11,9980.0
46,2021-01-18,10673.0
47,2021-01-25,1301.0


## Population data

In [13]:
pop_raw = pd.read_csv('data/nga_pop_adm1_2016.csv')
pop = pop_raw.copy()

In [14]:
# replace the FCT - makes joins easier
# pop = pop.replace('Federal Capital Territory', 'Abuja')

In [15]:
# rename the columns
pop = pop.rename(columns={'admin1Name_en': 'state_name',
                          'Unnamed: 1': 'state_code',
                          'Unnamed: 2': 'country',
                          'Unnamed: 3': 'country_code',
                          'Unnamed: 4': 'population'
                         }).sort_values(by='state_name')
pop.tail()

,state_name,state_code,country,country_code,population
8,Rivers,NG033,Nigeria,NG,7081412
33,Sokoto,NG034,Nigeria,NG,4879641
27,Taraba,NG035,Nigeria,NG,2984174
30,Yobe,NG036,Nigeria,NG,3197296
26,Zamfara,NG037,Nigeria,NG,4376911


In [16]:
# join with population an
state_cases_weekly = state_cases_weekly.set_index('REGION').join(
                        pop[['state_name', 'state_code', 'population']].set_index('state_name')) \
                        .reset_index().rename(columns={'index': 'state_name'})

state_cases_weekly['case_rate_100k'] = (state_cases_weekly['CASES']/state_cases_weekly['population'])*100000
state_cases_weekly = state_cases_weekly.drop(columns='population')


In [17]:
national_cases_weekly['case_rate_100k'] = (national_cases_weekly['CASES']/pop['population'].sum())*100000
national_cases_weekly.head()

,DATE,CASES,case_rate_100k
0,2020-03-02,1.0,0.000533
1,2020-03-09,1.0,0.000533
2,2020-03-16,1.0,0.000533
3,2020-03-23,33.0,0.017573
4,2020-03-30,95.0,0.050590


In [18]:
state_cases_weekly = state_cases_weekly.set_index('DATE').join(national_cases_weekly[['DATE','case_rate_100k']]\
                                                               .set_index('DATE'),rsuffix='_national').reset_index()

state_cases_weekly.tail()

,DATE,state_name,CASES,PREV_WEEK_CASES,weekly_change,state_code,case_rate_100k,case_rate_100k_national
1771,2021-01-25,Rivers,44.0,489.0,-91.002045,NG033,0.621345,0.69282
1772,2021-01-25,Sokoto,0.0,141.0,-100.000000,NG034,0.000000,0.69282
1773,2021-01-25,Taraba,0.0,68.0,-100.000000,NG035,0.000000,0.69282
1774,2021-01-25,Yobe,0.0,4.0,-100.000000,NG036,0.000000,0.69282
1775,2021-01-25,Zamfara,3.0,3.0,0.000000,NG037,0.068541,0.69282


In [19]:
state_cases_weekly['cases_vs_national'] = state_cases_weekly['case_rate_100k']-\
                                            state_cases_weekly['case_rate_100k_national']

In [20]:
state_cases_weekly['week'] = state_cases_weekly['DATE'].astype(str)

In [21]:
state_cases_weekly.head()

,DATE,state_name,CASES,PREV_WEEK_CASES,weekly_change,state_code,case_rate_100k,case_rate_100k_national,cases_vs_national,week
0,2020-03-02,Abia,0.0,0.0,0.0,NG001,0.0,0.000533,-0.000533,2020-03-02
1,2020-03-02,Adamawa,0.0,0.0,0.0,NG002,0.0,0.000533,-0.000533,2020-03-02
2,2020-03-02,Akwa Ibom,0.0,0.0,0.0,NG003,0.0,0.000533,-0.000533,2020-03-02
3,2020-03-02,Anambra,0.0,31.0,-100.0,NG004,0.0,0.000533,-0.000533,2020-03-02
4,2020-03-02,Bauchi,0.0,0.0,0.0,NG005,0.0,0.000533,-0.000533,2020-03-02


## Healthcare Data

In [22]:
# secondary and tertiary care only
health_raw = pd.read_csv('data/hospitals_nigeria.csv')
healthcare = health_raw.copy()

In [23]:
healthcare = healthcare.replace('FCT', 'Federal Capital Territory')
healthcare = healthcare.replace('Akwa-Ibom', 'Akwa Ibom')

In [24]:
healthcare.head()

,state,lga,ward,uid,facility_code,facility_name,reg_number,start_date,ownership,facility_level,longitude,latitude,operation_status,registration_status,license_status,created,last_updated
0,Abia,Aba North,Ariaria,48319756,01/01/1/1/2/0001,Aba Holy Wounds Hospital,NaN,2003-10-05,Private,Secondary,7.341340,5.11445,Operational,Registered,Licensed,2018-01-01 01:01:01,2019-05-16 09:46:23
1,Abia,Aba North,Umuola,47587317,01/01/1/1/2/0003,Alpha Inland Hospital,NaN,1985-04-02,Private,Secondary,7.385681,5.129272461,Operational,Registered,Licensed,2018-01-01 01:01:01,2020-05-22 13:41:36
2,Abia,Aba North,Old GRA,38145837,01/01/1/1/2/0004,Angels of Mercy Hospital,NaN,2010-12-10,Private,Secondary,7.354919,5.124084473,Operational,Registered,Licensed,2018-01-01 01:01:01,2019-05-26 14:52:35
3,Abia,Aba North,Old GRA,81463501,01/01/1/1/2/0006,Austin Graces Hospital,NaN,1993-03-05,Private,Secondary,7.362671,5.117126465,Operational,Registered,Licensed,2018-01-01 01:01:01,2020-05-22 13:45:38
4,Abia,Aba North,Asaokpuaja,25705792,01/01/1/1/2/0007,Berma Hospital,NaN,1986-05-08,Private,Secondary,7.348877,5.107299805,Operational,Registered,Licensed,2018-01-01 01:01:01,2020-05-27 16:59:38


In [25]:
healthcare_by_state = healthcare.groupby(['state'])['uid'].count()\
                        .reset_index().rename(columns={'uid': 'facilities'})

In [26]:
healthcare_by_state = healthcare_by_state.set_index('state').join(
                        pop[['state_name', 'state_code', 'population']].set_index('state_name')) \
                        .reset_index().rename(columns={'index': 'state_name'})

In [27]:
healthcare_by_state['pop_per_facility'] = (healthcare_by_state['population']/healthcare_by_state['facilities']).astype(int)

In [28]:
healthcare_by_state['fac_per_100k'] = (healthcare_by_state['facilities']/healthcare_by_state['population'])*100000

In [29]:
healthcare_by_state.sort_values(by='pop_per_facility').tail()

,state,facilities,state_code,population,pop_per_facility,fac_per_100k
4,Bauchi,69,NG005,6386388,92556,1.080423
36,Zamfara,42,NG037,4376911,104212,0.959581
35,Yobe,27,NG036,3197296,118418,0.844464
20,Katsina,45,NG021,7645575,169901,0.588576
17,Jigawa,32,NG018,5690516,177828,0.562339


## Produce the maps

In [30]:
with urlopen('https://development-data-hub-s3-public.s3.amazonaws.com/ddhfiles/145474/ngaadmbndaadm1osgof20161215.geojson') as response:
    nigeria_map = json.load(response)

In [31]:
fig = px.choropleth(state_cases_weekly,
                     geojson=nigeria_map,
                     featureidkey="properties.admin1Pcod",
                     locations='state_code',
                     color='case_rate_100k',
                     hover_name = 'state_name',
                     hover_data = {'state_code': False,
                                   'week' : False,
                                   'case_rate_100k': ':.2f',
                                   'case_rate_100k_national': ':.2f',
                                   'CASES': ':.0f',
                                   'PREV_WEEK_CASES': ':.0f',
                                   'weekly_change': ':.0f',
                                  },
                     animation_frame = 'week',
                     labels = {'CASES': '<br><b>Cases this week</b>',
                               'PREV_WEEK_CASES': '<i>Cases in previous week</i>',
                               'weekly_change': '<i>Weekly change</i>',
                               'case_rate_100k': '<b>Case rate (per 100k)</b>',
                               'case_rate_100k_national': '<i>National case rate (per 100k)</i>'
                              },
                      color_continuous_scale="thermal",
                      range_color=[0, 20],
    #                            opacity=0.75,
    #                            zoom=5, center = {"lat": 9.3247, "lon": 8.4548},
    #                            mapbox_style="carto-positron",
                     title = '<b>Covid-19 cases and healthcare in Nigeria'
                    )

fig.add_trace(px.scatter_geo(healthcare_by_state,
                             geojson=nigeria_map,
                             featureidkey="properties.admin1Pcod",
                             locations='state_code',
                             size='fac_per_100k',
                             hover_data = {'fac_per_100k' : ':.2f',
                                           'state_code' : False,
                                           'facilities': ':.0f',
                                           'pop_per_facility': ':.0f'
                                          },
                             labels = {'facilities': 'Healthcare facilities (secondary and tertiary)',
                                       'fac_per_100k': 'Facilities per 100k',
                                       'pop_per_facility': 'Population per facility'
                                      }
                            ).data[0])

fig.update_traces(dict(marker_line_width=1, marker_line_color="white"))

# Adjust map layout stylings
fig.update_geos(
    resolution=110,
    showcoastlines=True, coastlinecolor="white",
    showland=True, landcolor="LightGrey",
    showocean=False, oceancolor="LightBlue",
    showlakes=False, lakecolor="Blue",
    showrivers=True, rivercolor="white",
    fitbounds="locations"
)

fig.update_layout(height=800, hoverlabel=dict(bgcolor="black", font_size=12, font_family="Calibri"))

fig.show()

In [34]:
fig.write_html('covid-cases-map.html')